# Cybos Plus API 연습
---
## 주의
* 증권사 api 는 python 32 bit 환경에서만 돌아감.
* 32 bit jupyter 를 반드시 관리자 권한으로 실행

## 연결 상태 확인
* cybos plus 에 로그인 해야 가능한듯?

In [34]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos") # CyUil 에 있는 CpCybos 클래스 기져와서 객체 생성
print('결과 : ' + str(instCpCybos.IsConnect)) # 연결되면 1, 안되면 0

결과 : 1


## 주식코드 조회
---
* 상장된 종목 데이터 다루기

In [29]:
stockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(stockCode.GetCount()) # 상장된 모든 종목 수
print(stockCode.GetData(1, 0)) # 첫 번째 인자의 값이 0이면 종목 코드를, 1이면 종목명을, 2면 FullCode를 리턴합니다.

# code 찾기
code_naver = stockCode.NameToCode('NAVER')
index_naver = stockCode.CodeToIndex(code_naver)
print(code_naver, index_naver) # 종목이름으로 code 와 code index 검색

3167
동화약품
A035420 886


## 종목코드 가져오기

In [44]:
cpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr') # CpCodeMgr 클래스 객체 생성
codeList = cpCodeMgr.GetStockListByMarket(1) # 인자는 시장구분, 인자가 1 이면 유가증권시장
#print(codeList)

# 딕셔너리로 이름과 코드 저장
kospi = {}
for code in codeList:
    name = cpCodeMgr.CodeToName(code)
    kospi[code] = name
    pass

# csv 로 저장
f = open('c:\\Users\\김민수\\Desktop\\stock.csv', 'w')
for key, value in kospi.items():
    f.write("%s,%s\n" % (key, value))
f.close()

## 과거 데이터 가져오기
---
* StockChart 클래스는 RQ/RP 통신 방식
* 다른 통신방식으로 SB/PB 방식도 있음

In [49]:
stockChart = win32com.client.Dispatch('CpSysDib.StockChart')
stockChart.SetInputValue(0, 'A003540') # 요청하려는 데이터 셋팅, 첫번째 인자는 타입(0 은 종목코드), 두번쨰 인자는 값,
stockChart.SetInputValue(1, ord('2')) # 1 은 해당 종목조회방식을 지정, 1 은 기간, 2 는 갯수
stockChart.SetInputValue(4, 10) # 4 는 요청갯수, 일(day) 단위
stockChart.SetInputValue(5, 5) # 5 는 조회데이터 종류, 뒤 인자 5 는 종가데이터
stockChart.SetInputValue(6, ord('D')) # 6 은 차트 종류
stockChart.SetInputValue(9, ord('1'))

# blockRequest 는 서버에서 응답이 오기전까지 대기한다.
stockChart.BlockRequest()
numData = stockChart.GetHeaderValue(3)
print(numData)

# 위에서 요청한 10일간 종가데이터 조회
for i in range(numData):
    print(stockChart.GetDataValue(0, i))

10
13100
13150
13050
12950
13150
13450
13450
13400
13900
13950


In [59]:
stockChart = win32com.client.Dispatch('CpSysDib.StockChart')
stockChart.SetInputValue(0, 'A003540')
stockChart.SetInputValue(1, ord('2'))
stockChart.SetInputValue(4, 10)
stockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8)) # (일자, 시가, 고가, 저가, 종가, 거래량) 요청
stockChart.SetInputValue(6, ord('D'))
stockChart.SetInputValue(9, ord('1'))

stockChart.BlockRequest()
numData = stockChart.GetHeaderValue(3)
numField = stockChart.GetHeaderValue(1)
print(numData)

# 6 개의 데이터를 중첩 for 문으로 모두 출력
for i in range(numData):
    for d in range(numField):
        print(stockChart.GetDataValue(d, i), end=' ')
    print('')

10
20190712 13200 13200 13050 13100 41145 
20190711 13050 13200 12950 13150 98833 
20190710 13000 13100 12850 13050 101875 
20190709 13100 13250 12950 12950 91520 
20190708 13400 13400 13050 13150 179265 
20190705 13450 13550 13300 13450 164548 
20190704 13400 13500 13250 13450 111843 
20190703 13900 13900 13250 13400 249028 
20190702 13950 13950 13750 13900 78153 
20190701 13900 14050 13800 13950 131590 


# bookmark
https://wikidocs.net/3685